In [1]:
import pandas as pd
pd.set_option('display.width', 1000)
import geopandas as gpd
from pathlib import Path

In [25]:
dir = Path(".")

# get species crosswalk table
crosswalk = pd.read_csv('tbl/crosswalk.csv')

# list shapefiles by category and print the filenames
shp_dict = {}

shp_dict["birds"] = list(dir.glob("**/*Bird*.shp"))
shp_dict["mammals"] = list(dir.glob("**/*Mammal*.shp"))
shp_dict["amphibians"] = list(dir.glob("**/*Amphibian*.shp"))
shp_dict["SGCN"] = list(dir.glob("**/*SGCN Richness*.shp"))

count = 0

for category in shp_dict.keys():
    print(category, ":")
    for path in shp_dict[category]:
        print(path.name)
        count = count+1
    print("\n")
print(f"\nTotal of {count} shapefiles found.")

birds :
High Bird Richness.shp
Moderate Bird Richness.shp
Low Bird Richness.shp
SGCN Bird Richness.shp
Declining Birds Richness.shp
ModHigh Bird Richness.shp
High_Bird_Richness.shp
Moderate_Bird_Richness.shp
Declining_Birds_Richness.shp
SGCN_Bird_Richness.shp
Low_Bird_Richness.shp
ModHigh_Bird_Richness.shp


mammals :
Moderate Mammals Richness.shp
High Mammal Richness.shp
Low Mammal Richness.shp
SGCN Mammal Richness.shp
ModHigh_Mammal_Richness.shp
Declining Mammal Richness.shp
Low_Mammal_Richness.shp
SGCN_Mammal_Richness.shp
Moderate_Mammals_Richness.shp
Declining_Mammal_Richness.shp
ModHigh_Mammal_Richness.shp
High_Mammal_Richness.shp


amphibians :
Moderate Amphibian Richness.shp
ModHigh Amphibian Richness.shp
SGCN Amphibian Richness.shp
ModHigh_Amphibian_Richness.shp
Moderate_Amphibian_Richness.shp
SGCN_Amphibian_Richness.shp


SGCN :
SGCN Richness.shp



Total of 31 shapefiles found.


In [3]:
# we need to correct some of the species IDs
# use a dict with the incorrect ID as the key and correct ID as the value
# current as of 5/21/23, confirmed with Andy Baltensperger

id_dict = {
    #birds:
    "ameri_kest" : "am_kestrel",
    "am_pipit" : "amer_pipit",
    "am_redstrt" : "amer_redst",
    "bank_swall" : "bank_swalo",
    "bnk_swalow" : "bank_swalo",
    "barn_swall" : "barn_swalo",
    "brn_swalow" : "barn_swalo",
    "bb_sandpip" : "bb_sndpipr",
    "black_swif" : "blak_swift",
    "bartail_go" : "bt_godwit",
    "chba_chkde" : "cb_chicdee",
    "chp_sparow" : "ch_sparrow",
    "cmn_redpol" : "com_redpol",
    "gh_chickad" : "gh_chicade",
    "gldcrn_kin" : "gc_kinglet",
    "hering_gul" : "herng_gull",
    "her_thrush" : "hrmt_thrsh",
    "l_yelolegs" : "les_yelolg",
    "les_yelowl" : "les_yelolg",
    "ocr_warblr" : "oc_warbler",
    "os_flycatc" : "os_flyctch",
    "pac_wren" : "pacif_wren",
    "pine_siskn" : "pin_siskin",
    "ps_flyctch" : "ps_flycach",
    "red_phalar" : "red_phlrop",
    "rn_phalaro" : "rn_phalrop",
    "rst_blkbrd" : "ru_blckbrd",
    "ruf_huming" : "ruf_humbrd",
    "rw_blkbird" : "rw_blakbrd",
    "sa_sparrow" : "sav_sparow",
    "sb_dowitch" : "sb_dowichr", 
    "shrtearowl" : "shtear_owl",
    "shrter_owl" : "shtear_owl",
    "song_sparr" : "sng_sparow",
    "swa_thrush" : "swn_thrush",
    "tre_swalow" : "tree_swalo",
    "tree_swall" : "tree_swalo",
    "yb_loon" : "yelobil_ln",
    "yebil_loon" : "yelobil_ln",
    "yelo_wrblr" : "ye_warbler",
    #amphibians
    "lt_salamnd" : "longtoedsa",
    "nwsalamndr" : "nwsalamand",
    "redlg_frog" : "redleggedf",
    "rufskn_nwt" : "roughskinn",
    "west_toad" : "westerntoa",
}

In [4]:
# create a function that will rename columns if they match keys in the dict

def rename_species(gdf, id_dict, crs):
    for col in gdf.columns:
        try:
            new_col = id_dict[col]
            gdf.rename(columns={col : new_col}, inplace=True)
            # if columns are renamed, theres a chance for duplicate column names...
            # drop any dup columns by name, keeping the first record by default
            gdf = gpd.GeoDataFrame(gdf.T.drop_duplicates().T, geometry="geometry", crs=crs)
        except:
            pass
    return gdf

In [5]:
# create a function that will list all unique species in each category
# and also return a list of geodataframes by category (EPSG 3338)
# this is also where species names can be corrected!

def list_species_by_category(shp_dict, category):

    crs = "EPSG:3338"

    names = []
    polys = []
    for shp in shp_dict[category]:
        names.append(shp.name)
        polys.append(gpd.read_file(shp).to_crs(crs))        
    
    species = []
    polys_renamed = []
    for poly in polys:
        poly_rn = rename_species(poly, id_dict, crs)
        polys_renamed.append(poly_rn)
        for col in poly_rn.columns:
            if col not in ['HUC_8', 'HUC_10', 'HUC_12', 'SppRichnes', 'geometry']:
                species.append(col)
                
    return list(set(species)), polys_renamed, names

In [6]:
# list em

birds, birds_gdfs, birds_shps = list_species_by_category(shp_dict, "birds")
mammals, mammals_gdfs, mammals_shps = list_species_by_category(shp_dict, "mammals")
amphibians, amphibians_gdfs, amphibians_shps = list_species_by_category(shp_dict, "amphibians")
sgcn, sgcn_gdfs, sgcn_shps = list_species_by_category(shp_dict, "SGCN")

In [8]:
# count em

print("Total unique bird species in shapefiles: ", len(birds))
print("Total unique mammal species in shapefiles: ", len(mammals))
print("Total unique amphibian species in shapefiles: ", len(amphibians))
print("Total unique terrestrial SGCN species in shapefiles: ", len(sgcn))
print("\n")
print("Sum of unique bird, mammal, amphibian species counts in shapefiles: ", (len(birds) + len(mammals) + len(amphibians)))
print("\n")
print("Total unique terrestrial SGCN species referenced in report: 268")

Total unique bird species in shapefiles:  159
Total unique mammal species in shapefiles:  45
Total unique amphibian species in shapefiles:  7
Total unique terrestrial SGCN species in shapefiles:  206


Sum of unique bird, mammal, amphibian species counts in shapefiles:  211


Total unique terrestrial SGCN species referenced in report: 268


In [8]:
# optionally, export species lists as CSVs for easier viewing / crosswalk creation
for species_list, name in zip([birds, mammals, amphibians, sgcn], ["birds", "mammals", "amphibians", "sgcn"]):
     pd.DataFrame(data=species_list, columns=['species']).sort_values(by='species').to_csv(f"tbl/{name}.csv", index=False)

In [9]:
# optionally, rewrite the shapefiles to incorporate any renamed columns
# these are the "clean" copies that should be hosted by SNAP 
export_dir = Path('/Users/joshpaul/species/export/to_host')

all_shps = birds_shps + mammals_shps + amphibians_shps + sgcn_shps
all_gdfs = birds_gdfs + mammals_gdfs + amphibians_gdfs + sgcn_gdfs

for shp, gdf in zip(all_shps, all_gdfs):
    outfile = Path.joinpath(export_dir, ("_").join(shp.split(" ")))
    print(f"Saving {outfile}...")
    gdf.to_file(outfile)


Saving /Users/joshpaul/species/export/to_host/High_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Moderate_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Low_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/SGCN_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Declining_Birds_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/ModHigh_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/High_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Moderate_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Declining_Birds_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/SGCN_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Low_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/ModHigh_Bird_Richness.shp...
Saving /Users/joshpaul/species/export/to_host/Moderate_Mammals_Richness.shp...
Saving /Users/joshpaul/species/ex

In [9]:
print(sgcn)
for gdf, shp in zip(sgcn_gdfs, sgcn_shps): 
    print(shp)    
    print(gdf.head())

['westerntoa', 'ps_flycach', 'tree_swalo', 'sm_longspr', 'pi_grosbek', 'gold_eagle', 'boreal_owl', 'gyrfalcon', 'bl_gilemot', 'red_crsbil', 'whimbrel', 'aktny_shrw', 'root_vole', 'borl_chkde', 'grwf_goose', 'dusky_shrw', 'mac_warblr', 'mbl_murlet', 'polar_bear', 'hrmt_thrsh', 'ag_squirel', 'nwsalamand', 'blklgd_kit', 'rn_grebe', 'colard_pik', 'ringd_seal', 'pac_walrus', 'ruf_humbrd', 'no_shrike', 'shtear_owl', 'dovekie', 'rb_sapsckr', 'red_knot', 'we_sndpipr', 'cas_auklet', 'kit_murlet', 'snowy_owl', 'empr_goose', 'par_auklet', 'sb_dowichr', 'bald_eagle', 'kingfisher', 'ncol_lemng', 'os_flyctch', 'gc_rsyfnch', 'pygmy_shrw', 'pin_siskin', 'no_wheater', 'ca_myotis', 'nor_hawk_o', 'roughskinn', 'lt_jaeger', 'cack_goose', 'la_longspr', 'wsk_auklet', 'amwtr_shrw', 'dunlin', 'fox_sparow', 'c_yelothrt', 'cre_auklet', 'spec_eider', 'ch_sparrow', 'pacif_wren', 'li_sparrow', 'rf_cormrnt', 'sp_sndpipr', 'brown_cree', 'swn_thrush', 'rl_kitiwak', 'pi_guilmot', 'lesr_scaup', 'redleggedf', 'bt_woodra

In [10]:
print(birds)
for gdf, shp in zip(birds_gdfs, birds_shps): 
    print(shp)    
    print(gdf.head())

['ps_flycach', 'tree_swalo', 'sm_longspr', 'pi_grosbek', 'gold_eagle', 'boreal_owl', 'gyrfalcon', 'bl_gilemot', 'red_crsbil', 'whimbrel', 'borl_chkde', 'grwf_goose', 'mac_warblr', 'mbl_murlet', 'hrmt_thrsh', 'rn_grebe', 'blklgd_kit', 'ruf_humbrd', 'no_shrike', 'shtear_owl', 'rb_sapsckr', 'dovekie', 'red_knot', 'we_sndpipr', 'cas_auklet', 'kit_murlet', 'snowy_owl', 'empr_goose', 'sb_dowichr', 'par_auklet', 'bald_eagle', 'kingfisher', 'os_flyctch', 'gc_rsyfnch', 'pin_siskin', 'no_wheater', 'nor_hawk_o', 'lt_jaeger', 'wsk_auklet', 'cack_goose', 'la_longspr', 'dunlin', 'fox_sparow', 'c_yelothrt', 'cre_auklet', 'wc_sparrow', 'spec_eider', 'ch_sparrow', 'pacif_wren', 'li_sparrow', 'rf_cormrnt', 'sp_sndpipr', 'swn_thrush', 'brown_cree', 'rl_kitiwak', 'pi_guilmot', 'lesr_scaup', 'sno_buntng', 'var_thrush', 'blcp_chkde', 'amer_pipit', 'peal_falcn', 'am_kestrel', 'gh_chicade', 'to_warbler', 'no_harrier', 'brthi_curl', 'thbi_murre', 'bt_godwit', 'ha_wdpeckr', 'bb_plover', 'ye_warbler', 'hud_godwi

In [11]:
print(mammals)
for gdf, shp in zip(mammals_gdfs, mammals_shps): 
    print(shp)    
    print(gdf.head())

['ak_hare', 'beard_seal', 'spotd_seal', 'woodchuck', 'snosh_hare', 'root_vole', 'aktny_shrw', 'sihair_bat', 'nbog_lemng', 'nrdbck_vol', 'dusky_shrw', 'lgtail_vol', 'bt_woodrat', 'polar_bear', 'ag_squirel', 'colard_pik', 'nw_dermous', 'pac_walrus', 'ringd_seal', 'st_sealion', 'lole_mytis', 'nfl_squirl', 'meadow_vol', 'nbrn_lemng', 'tndra_shrw', 'red_squirl', 'keen_mytis', 'alxdr_wolf', 'sredbk_vol', 'arctic_fox', 'bargd_shrw', 'pahbr_seal', 'mjmp_mouse', 'lilb_mytis', 'n_fur_seal', 'harbr_seal', 'ak_marmot', 'ncol_lemng', 'ciner_shrw', 'taiga_vole', 'pygmy_shrw', 'ca_myotis', 'sing_vole', 'hry_marmot', 'amwtr_shrw']
Moderate Mammals Richness.shp
      HUC_8      HUC_10        HUC_12  nbog_lemng  st_sealion  woodchuck  SppRichnes                                           geometry
0  19030102  1903010215  190301021506           0           1          0           1  POLYGON ((-979806.523 441870.783, -979752.322 ...
1  19030102  1903010214  190301021406           0           1          0   

In [12]:
print(amphibians)
for gdf, shp in zip(amphibians_gdfs, amphibians_shps): 
    print(shp)    
    print(gdf.head())

['westerntoa', 'longtoedsa', 'columbiasp', 'roughskinn', 'nwsalamand', 'woodfrog', 'redleggedf']
Moderate Amphibian Richness.shp
      HUC_8      HUC_10        HUC_12  SppRichnes  roughskinn                                           geometry
0  19030102  1903010215  190301021506           0           0  POLYGON ((-979806.523 441870.783, -979752.322 ...
1  19030102  1903010214  190301021406           0           0  POLYGON ((-987830.627 440768.120, -988047.792 ...
2  19030102  1903010215  190301021505           0           0  POLYGON ((-969780.978 442087.182, -969798.483 ...
3  19030102  1903010214  190301021405           0           0  MULTIPOLYGON (((-991838.223 445056.182, -99188...
4  19030102  1903010214  190301021404           0           0  POLYGON ((-969970.132 455008.363, -969933.319 ...
ModHigh Amphibian Richness.shp
      HUC_8      HUC_10        HUC_12  woodfrog  westerntoa  columbiasp  longtoedsa  nwsalamand  SppRichnes                                           geometry
0  

In [38]:
# use the SGCN gdf to export a basic HUC12 polygon layer; this can be used for spatial lookup of the HUC codes

sgcn_gdfs[0].drop(columns=["HUC_8", "HUC_10"])[["HUC_12", "geometry"]].to_crs('EPSG:4326').to_file('export/huc12_species.shp')

In [18]:
# melt all gdfs together and return a long format table indexed by HUC and species type

def melt_gdfs_and_concatenate(gdfs, types):
    to_concat = []
    for gdf, type in zip(gdfs, types):
        gdf["type"] = type
        melted = pd.melt(gdf.drop(columns=["geometry", "SppRichnes", "HUC_8", "HUC_10"], axis=0), id_vars=["HUC_12", "type"])
        melted.drop_duplicates(inplace=True)
        to_concat.append(melted.where(melted["value"] != 0).dropna())
    
    return pd.concat(to_concat)
    

In [20]:
types = ["birds", "mammals", "amphibians"]
gdfs = [birds_gdfs[3], mammals_gdfs[3], amphibians_gdfs[2]] # full sgcn lists only, not using concern level for now

lookup_df = melt_gdfs_and_concatenate(gdfs, types).reset_index(drop=True)
lookup_df

,HUC_12,type,variable,value
0,190207021608,birds,al_ck_goos,1
1,190101030206,birds,al_flyctch,1
2,190101030203,birds,al_flyctch,1
3,190102032106,birds,al_flyctch,1
4,190102040501,birds,al_flyctch,1
...,...,...,...,...
1134489,190103040208,amphibians,longtoedsa,1.0
1134490,190103040201,amphibians,longtoedsa,1.0
1134491,190103040105,amphibians,longtoedsa,1.0
1134492,190103040102,amphibians,longtoedsa,1.0


In [21]:
# export the lookup df as csv (2 copies, one for repo and one for export)
lookup_df.rename(columns={"variable" : "species_ID"}, inplace=True)
lookup_df[["HUC_12", "type", "species_ID"]].to_csv("tbl/huc12_species_lookup.csv", index=False)
lookup_df[["HUC_12", "type", "species_ID"]].to_csv("export/huc12_species_lookup.csv", index=False)

# add a dummy geometry column and export as shapefile
lookup_df['lat'] = 64.84
lookup_df['lon'] = -147.72

lookup_gdf = gpd.GeoDataFrame(lookup_df[["HUC_12", "type", "species_ID"]], geometry=gpd.points_from_xy(lookup_df.lon, lookup_df.lat), crs="EPSG:4326")
lookup_gdf.to_file('export/huc12_species_lookup.shp')

In [22]:
lookup_gdf.head()

,HUC_12,type,species_ID,geometry
0,190207021608,birds,al_ck_goos,POINT (-147.72000 64.84000)
1,190101030206,birds,al_flyctch,POINT (-147.72000 64.84000)
2,190101030203,birds,al_flyctch,POINT (-147.72000 64.84000)
3,190102032106,birds,al_flyctch,POINT (-147.72000 64.84000)
4,190102040501,birds,al_flyctch,POINT (-147.72000 64.84000)


In [35]:
# proof of concept function to pull species data given any HUC 12 ID as input
# and get full names using crosswalk table

def combine_names(common_names, sci_names):
    combined = []
    for c, s in zip(common_names, sci_names):
        combined.append(f"{c} ({s})")
    # drop duplicates on return
    return list(set(combined))

def get_species_by_huc(huc, lookup_df, print_it=True):
    sub_df = lookup_df[lookup_df["HUC_12"]==huc]
    sub_df = sub_df.merge(crosswalk, how='left', left_on="species_ID", right_on="species_id")

    huc_birds_common = list(sub_df[sub_df["type"] == "birds"]["common_name"].tolist())
    huc_birds_sci = list(sub_df[sub_df["type"] == "birds"]["scientific_name"].tolist())
    huc_birds = combine_names(huc_birds_common, huc_birds_sci)

    huc_mammals_common = list(sub_df[sub_df["type"] == "mammals"]["common_name"].tolist())
    huc_mammals_sci = list(sub_df[sub_df["type"] == "mammals"]["scientific_name"].tolist())
    huc_mammals = combine_names(huc_mammals_common, huc_mammals_sci)

    huc_amphibs_common = list(sub_df[sub_df["type"] == "amphibians"]["common_name"].tolist())
    huc_amphibs_sci = list(sub_df[sub_df["type"] == "amphibians"]["scientific_name"].tolist())
    huc_amphibs = combine_names(huc_amphibs_common, huc_amphibs_sci)

    if print_it == True:
        print(f"Modeled habitat data indicate that the following species of greatest conservation need are present in HUC {huc}:")
        print("\n")
        print(f"Birds: {huc_birds}")
        print("\n")
        print(f"Mammals: {huc_mammals}")
        print("\n")
        print(f"Amphibians: {huc_amphibs}")

    return huc_birds, huc_mammals, huc_amphibs

In [36]:
huc = '190103010713'
huc_birds_, huc_mammals_, huc_amphibs_ = get_species_by_huc(huc, lookup_df)

Modeled habitat data indicate that the following species of greatest conservation need are present in HUC 190103010713:


Birds: ['Western Screech Owl (Megascops kennicottii)', 'Belted Kingfisher (Megaceryle alcyon)', 'Northern Flicker (Colaptes auratus)', 'Ancient Murrelet (Synthliboramphus antiquus)', 'Northern Hawk-Owl (Surnia ulula)', 'Rusty Blackbird (Euphagus carolinus)', 'Bohemian Waxwing (Bombycilla garrulus)', 'Hairy Woodpecker (Picoides villosus)', 'Pacific Golden Plover (Pluvialis fulva)', 'Pacific Loon (Gavia pacifica)', 'Trumpeter Swan (Cygnus buccinator)', 'Bald Eagle (Haliaeetus leucocephalus)', 'Brown Creeper (Certhia americana)', 'Savannah Sparrow (Passerculus sandwichensis)', "Wilson's Warbler (Cardellina pusilla)", 'Greater White-fronted Goose (Anser albifrons)', 'Lesser Scaup (Aythya affinis)', 'Surfbird (Calidris virgata)', 'Common Yellowthroat (Geothlypis trichas)', 'Pigeon Guillemot (Cepphus columba)', 'Black Oystercatcher (Haematopus bachmani)', "Peale's Falcon 

In [26]:
crosswalk

,species_id,common_name,scientific_name
0,columbiasp,Columbia Spotted Frog,Rana luteiventris
1,longtoedsa,Long-toed Salamander,Ambystoma macrodactylum
2,nwsalamand,Northwestern Salamander,Ambystoma gracile
3,redleggedf,Red-legged Frog,Rana aurora
4,roughskinn,Rough-skinned Newt,Taricha granulosa
...,...,...,...
206,wsk_auklet,Whiskered Auklet,Aethia pygmaea
207,ww_scoter,White-winged Scoter,Melanitta deglandi
208,ye_warbler,Yellow Warbler,Setophaga petechia
209,yelobil_ln,Yellow-billed Loon,Gavia adamsii
